In [1]:
import pandas as pd
import numpy as np
import os
import time
import pickle
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

## Login

In [2]:
synergy_url = 'https://www.synergysportstech.com'

In [3]:
def login(driver, cred, synergy_url=synergy_url):
    driver.get(f'{synergy_url}/synergy/')
    # Get username & password
    with open(cred, 'r') as f:
        user_password = f.readlines()[0].replace('\n', '').split(', ')
    # Login
    userfield = driver.find_element_by_name('txtUserName')
    userfield.send_keys(user_password[0])
    passwordfield = driver.find_element_by_name('txtPassword')
    passwordfield.send_keys(user_password[1])
    driver.find_element_by_name('btnLogin').click()

In [4]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/xulian/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


In [6]:
# driver = webdriver.Chrome('/usr/local/bin/chromedriver')

In [7]:
cred = 'synergy_cred.txt'
login(driver, cred)

/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_51688/1425548842.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  userfield = driver.find_element_by_name('txtUserName')
/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_51688/1425548842.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  passwordfield = driver.find_element_by_name('txtPassword')
/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_51688/1425548842.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name('btnLogin').click()


## Basic Functions

In [8]:
def read_html(filename):
    soup = BeautifulSoup(open(filename), "html.parser")
    return soup

In [9]:
def dump_pickle(file, filename):
    outfile = open(filename, 'wb')
    pickle.dump(file, outfile)
    outfile.close()

def load_pickle(filename):
    infile = open(filename, 'rb')
    file = pickle.load(infile)
    infile.close()
    return file

## Download Team Stats Sheets

In [10]:
side_dict = {'offense': '1', 'defense': '0'}

In [11]:
def download_team_stats(directory, team_id, season_id, side,
                        synergy_url=synergy_url, side_dict=side_dict, driver=driver):
    bool_side = side_dict[side]
    tm_stat_url = (
        f'{synergy_url}/Synergy/Sport/Basketball/web/teamsst/Video/QuantifiedTeam2Printable.aspx?'
        f'iSeasonID={season_id}&offensive={bool_side}&pergame=0&iTeamID={team_id}'
    )
    driver.get(tm_stat_url)
    with open(f'{directory}/{team_id}_{side}.html', 'w') as f:
        f.write(driver.page_source)

In [12]:
def download_cumulative_stats(directory, team_id, season_id,
                              synergy_url=synergy_url, driver=driver):
    cumulative_stat_url = (
        f'{synergy_url}/Synergy/Sport/Basketball/web/teamsst/Video/ViewReport.aspx?'
        f'iSeasonID={season_id}&pergame=0&Type=CumulativeStats&iTeamID={team_id}'
    )
    driver.get(cumulative_stat_url)
    with open(f'{directory}/{team_id}_box.html', 'w') as f:
        f.write(driver.page_source)

In [13]:
def download_season(season_id, league_dict,
                    synergy_url=synergy_url, side_dict=side_dict, driver=driver):
    league_name = league_dict['league_name']
    print(league_name, season_id)
    directory = f'data_{league_name}_{season_id}'
    if directory not in os.listdir():
        os.mkdir(directory)
    for team_id in league_dict['seasons'][season_id].keys():
        print(team_id, league_dict['seasons'][season_id][team_id])
        for side in ['offense', 'defense']:
            download_team_stats(directory, team_id, season_id, side,
                                synergy_url=synergy_url, side_dict=side_dict, 
                                driver=driver)
        download_cumulative_stats(directory, team_id, season_id,
                                  synergy_url=synergy_url, driver=driver)
        time.sleep(3)
    print()
    print('Done.')

### Prepare Dict

In [14]:
def get_team_ids(season, synergy_url=synergy_url):
    driver.get(f'{synergy_url}/Synergy/Sport/Basketball/web/teamsst/Video/QuantifiedTeam2.aspx')
    
    # Select 
    leaguefield = driver.find_element_by_name('ctl00$MainContent$lstLeagues')
    leaguefield.send_keys('College Men')
    time.sleep(3)
    seasonfield = driver.find_element_by_name('ctl00$MainContent$lstSeasons')
    season_adj = f'{season} - {season+1}'
    seasonfield.send_keys(season_adj)
    time.sleep(3)
#     competitionfield = driver.find_element_by_name('ctl00$MainContent$lstCompetition')
#     competitionfield.send_keys('All')
#     time.sleep(3)
    divisionfield = driver.find_element_by_name('ctl00$MainContent$lstDivisionGroup')
    divisionfield.send_keys('NCAA Division I')
    time.sleep(3)
    
    # Get team ids
    team_dict = {}
    for row in driver.find_element_by_name('ctl00$MainContent$lstTeam').find_elements_by_tag_name('option'):
        if row.get_attribute('value') != '0':
            team_dict[row.get_attribute('value')] = row.text
    time.sleep(3)
            
    return team_dict

In [15]:
ncaa_dict = {'league_name': 'ncaa',
             'seasons': {season: get_team_ids(season)
                         for season in range(2021, 2022)}
            }
# ncaa_dict = load_pickle('ncaa_dict')

/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_51688/3138967872.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  leaguefield = driver.find_element_by_name('ctl00$MainContent$lstLeagues')
/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_51688/3138967872.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  seasonfield = driver.find_element_by_name('ctl00$MainContent$lstSeasons')
/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_51688/3138967872.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  divisionfield = driver.find_element_by_name('ctl00$MainContent$lstDivisionGroup')
/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_51688/3138967872.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  for row in driver.find_element_by_name('ctl0

In [15]:
# dump_pickle(ncaa_dict, 'ncaa_dict')

### Download

#### 2016-17

In [14]:
download_season(2016, ncaa_dict)

ncaa 2016
1281 Abilene Christian Wildcats
184 Air Force Falcons
174 Akron Zips
691 Alabama A&M Bulldogs
34 Alabama Crimson Tide
714 Alabama State Hornets
203 Albany Great Danes
307 Alcorn State Braves
322 American University Eagles
851 Appalachian State Mountaineers
122 Arizona State Sun Devils
40 Arizona Wildcats
78 Arkansas Razorbacks
119 Arkansas State Red Wolves
215 Arkansas-(Little Rock) Trojans
431 Arkansas-(Pine Bluff) Golden Lions
305 Army Black Knights
160 Auburn Tigers
448 Austin Peay Governors
325 Ball State Cardinals
173 Baylor Bears
3285 Bellarmine Knights
196 Belmont Bruins
436 Bethune-Cookman Wildcats
863 Binghamton Bearcats
171 Boise State Broncos
43 Boston College Eagles
438 Boston University Terriers
445 Bowling Green Falcons
141 Bradley Braves
159 Brigham Young Cougars
294 Brown Bears
854 Bryant University Bulldogs
123 Bucknell Bison
339 Buffalo Bulls
90 Butler Bulldogs
434 Cal Poly SLO Mustangs
869 Cal State Bakersfield Roadrunners
335 Cal State Fullerton Titans
682

81 UNLV Rebels
80 USC Trojans
849 USC Upstate Spartans
167 Utah State Aggies
67 Utah Utes
666 Utah Valley Wolverines
186 UTEP Miners
65 Valparaiso University
133 Vanderbilt Commodores
293 Vermont Catamounts
137 Villanova Wildcats
175 Virginia Cavaliers
442 Virginia Commonwealth Rams
284 Virginia Military Keydets
154 Virginia Tech Hokies
686 Wagner Seahawks
45 Wake Forest Demon Deacons
118 Washington Huskies
94 Washington State Cougars
380 Weber State Wildcats
84 West Virginia Mountaineers
677 Western Carolina Catamounts
861 Western Illinois Leathernecks
73 Western Kentucky Hilltoppers
297 Western Michigan Broncos
194 Wichita State Shockers
456 William & Mary Tribe
195 Winthrop Eagles
47 Wisconsin Badgers
162 Wisconsin-(Green Bay) Phoenix
156 Wisconsin-(Milwaukee) Panthers
665 Wofford Terriers
387 Wright State Raiders
136 Wyoming Cowboys
169 Xavier Musketeers
139 Yale Bulldogs
326 Youngstown State Penguins

Done.


#### 2017-18

In [15]:
download_season(2017, ncaa_dict)

ncaa 2017
1281 Abilene Christian Wildcats
184 Air Force Falcons
174 Akron Zips
691 Alabama A&M Bulldogs
34 Alabama Crimson Tide
714 Alabama State Hornets
203 Albany Great Danes
307 Alcorn State Braves
322 American University Eagles
851 Appalachian State Mountaineers
122 Arizona State Sun Devils
40 Arizona Wildcats
78 Arkansas Razorbacks
119 Arkansas State Red Wolves
215 Arkansas-(Little Rock) Trojans
431 Arkansas-(Pine Bluff) Golden Lions
305 Army Black Knights
160 Auburn Tigers
448 Austin Peay Governors
325 Ball State Cardinals
173 Baylor Bears
3285 Bellarmine Knights
196 Belmont Bruins
436 Bethune-Cookman Wildcats
863 Binghamton Bearcats
171 Boise State Broncos
43 Boston College Eagles
438 Boston University Terriers
445 Bowling Green Falcons
141 Bradley Braves
159 Brigham Young Cougars
294 Brown Bears
854 Bryant University Bulldogs
123 Bucknell Bison
339 Buffalo Bulls
90 Butler Bulldogs
434 Cal Poly SLO Mustangs
869 Cal State Bakersfield Roadrunners
335 Cal State Fullerton Titans
682

81 UNLV Rebels
80 USC Trojans
849 USC Upstate Spartans
167 Utah State Aggies
67 Utah Utes
666 Utah Valley Wolverines
186 UTEP Miners
65 Valparaiso University
133 Vanderbilt Commodores
293 Vermont Catamounts
137 Villanova Wildcats
175 Virginia Cavaliers
442 Virginia Commonwealth Rams
284 Virginia Military Keydets
154 Virginia Tech Hokies
686 Wagner Seahawks
45 Wake Forest Demon Deacons
118 Washington Huskies
94 Washington State Cougars
380 Weber State Wildcats
84 West Virginia Mountaineers
677 Western Carolina Catamounts
861 Western Illinois Leathernecks
73 Western Kentucky Hilltoppers
297 Western Michigan Broncos
194 Wichita State Shockers
456 William & Mary Tribe
195 Winthrop Eagles
47 Wisconsin Badgers
162 Wisconsin-(Green Bay) Phoenix
156 Wisconsin-(Milwaukee) Panthers
665 Wofford Terriers
387 Wright State Raiders
136 Wyoming Cowboys
169 Xavier Musketeers
139 Yale Bulldogs
326 Youngstown State Penguins

Done.


#### 2018-19

In [16]:
download_season(2018, ncaa_dict)

ncaa 2018
1281 Abilene Christian Wildcats
184 Air Force Falcons
174 Akron Zips
691 Alabama A&M Bulldogs
34 Alabama Crimson Tide
714 Alabama State Hornets
203 Albany Great Danes
307 Alcorn State Braves
322 American University Eagles
851 Appalachian State Mountaineers
122 Arizona State Sun Devils
40 Arizona Wildcats
78 Arkansas Razorbacks
119 Arkansas State Red Wolves
215 Arkansas-(Little Rock) Trojans
431 Arkansas-(Pine Bluff) Golden Lions
305 Army Black Knights
160 Auburn Tigers
448 Austin Peay Governors
325 Ball State Cardinals
173 Baylor Bears
3285 Bellarmine Knights
196 Belmont Bruins
436 Bethune-Cookman Wildcats
863 Binghamton Bearcats
171 Boise State Broncos
43 Boston College Eagles
438 Boston University Terriers
445 Bowling Green Falcons
141 Bradley Braves
159 Brigham Young Cougars
294 Brown Bears
854 Bryant University Bulldogs
123 Bucknell Bison
339 Buffalo Bulls
90 Butler Bulldogs
434 Cal Poly SLO Mustangs
869 Cal State Bakersfield Roadrunners
335 Cal State Fullerton Titans
682

81 UNLV Rebels
80 USC Trojans
849 USC Upstate Spartans
167 Utah State Aggies
67 Utah Utes
666 Utah Valley Wolverines
186 UTEP Miners
65 Valparaiso University
133 Vanderbilt Commodores
293 Vermont Catamounts
137 Villanova Wildcats
175 Virginia Cavaliers
442 Virginia Commonwealth Rams
284 Virginia Military Keydets
154 Virginia Tech Hokies
686 Wagner Seahawks
45 Wake Forest Demon Deacons
118 Washington Huskies
94 Washington State Cougars
380 Weber State Wildcats
84 West Virginia Mountaineers
677 Western Carolina Catamounts
861 Western Illinois Leathernecks
73 Western Kentucky Hilltoppers
297 Western Michigan Broncos
194 Wichita State Shockers
456 William & Mary Tribe
195 Winthrop Eagles
47 Wisconsin Badgers
162 Wisconsin-(Green Bay) Phoenix
156 Wisconsin-(Milwaukee) Panthers
665 Wofford Terriers
387 Wright State Raiders
136 Wyoming Cowboys
169 Xavier Musketeers
139 Yale Bulldogs
326 Youngstown State Penguins

Done.


#### 2019-20

In [17]:
download_season(2019, ncaa_dict)

ncaa 2019
1281 Abilene Christian Wildcats
184 Air Force Falcons
174 Akron Zips
691 Alabama A&M Bulldogs
34 Alabama Crimson Tide
714 Alabama State Hornets
203 Albany Great Danes
307 Alcorn State Braves
322 American University Eagles
851 Appalachian State Mountaineers
122 Arizona State Sun Devils
40 Arizona Wildcats
78 Arkansas Razorbacks
119 Arkansas State Red Wolves
215 Arkansas-(Little Rock) Trojans
431 Arkansas-(Pine Bluff) Golden Lions
305 Army Black Knights
160 Auburn Tigers
448 Austin Peay Governors
325 Ball State Cardinals
173 Baylor Bears
3285 Bellarmine Knights
196 Belmont Bruins
436 Bethune-Cookman Wildcats
863 Binghamton Bearcats
171 Boise State Broncos
43 Boston College Eagles
438 Boston University Terriers
445 Bowling Green Falcons
141 Bradley Braves
159 Brigham Young Cougars
294 Brown Bears
854 Bryant University Bulldogs
123 Bucknell Bison
339 Buffalo Bulls
90 Butler Bulldogs
434 Cal Poly SLO Mustangs
869 Cal State Bakersfield Roadrunners
335 Cal State Fullerton Titans
682

81 UNLV Rebels
80 USC Trojans
849 USC Upstate Spartans
167 Utah State Aggies
67 Utah Utes
666 Utah Valley Wolverines
186 UTEP Miners
65 Valparaiso University
133 Vanderbilt Commodores
293 Vermont Catamounts
137 Villanova Wildcats
175 Virginia Cavaliers
442 Virginia Commonwealth Rams
284 Virginia Military Keydets
154 Virginia Tech Hokies
686 Wagner Seahawks
45 Wake Forest Demon Deacons
118 Washington Huskies
94 Washington State Cougars
380 Weber State Wildcats
84 West Virginia Mountaineers
677 Western Carolina Catamounts
861 Western Illinois Leathernecks
73 Western Kentucky Hilltoppers
297 Western Michigan Broncos
194 Wichita State Shockers
456 William & Mary Tribe
195 Winthrop Eagles
47 Wisconsin Badgers
162 Wisconsin-(Green Bay) Phoenix
156 Wisconsin-(Milwaukee) Panthers
665 Wofford Terriers
387 Wright State Raiders
136 Wyoming Cowboys
169 Xavier Musketeers
139 Yale Bulldogs
326 Youngstown State Penguins

Done.


#### 2020-21

In [18]:
download_season(2020, ncaa_dict)

ncaa 2020
1281 Abilene Christian Wildcats
184 Air Force Falcons
174 Akron Zips
691 Alabama A&M Bulldogs
34 Alabama Crimson Tide
714 Alabama State Hornets
203 Albany Great Danes
307 Alcorn State Braves
322 American University Eagles
851 Appalachian State Mountaineers
122 Arizona State Sun Devils
40 Arizona Wildcats
78 Arkansas Razorbacks
119 Arkansas State Red Wolves
215 Arkansas-(Little Rock) Trojans
431 Arkansas-(Pine Bluff) Golden Lions
305 Army Black Knights
160 Auburn Tigers
448 Austin Peay Governors
325 Ball State Cardinals
173 Baylor Bears
3285 Bellarmine Knights
196 Belmont Bruins
863 Binghamton Bearcats
171 Boise State Broncos
43 Boston College Eagles
438 Boston University Terriers
445 Bowling Green Falcons
141 Bradley Braves
159 Brigham Young Cougars
854 Bryant University Bulldogs
123 Bucknell Bison
339 Buffalo Bulls
90 Butler Bulldogs
434 Cal Poly SLO Mustangs
869 Cal State Bakersfield Roadrunners
335 Cal State Fullerton Titans
682 Cal State Northridge Matadors
1036 Californi

137 Villanova Wildcats
175 Virginia Cavaliers
442 Virginia Commonwealth Rams
284 Virginia Military Keydets
154 Virginia Tech Hokies
686 Wagner Seahawks
45 Wake Forest Demon Deacons
118 Washington Huskies
94 Washington State Cougars
380 Weber State Wildcats
84 West Virginia Mountaineers
677 Western Carolina Catamounts
861 Western Illinois Leathernecks
73 Western Kentucky Hilltoppers
297 Western Michigan Broncos
194 Wichita State Shockers
456 William & Mary Tribe
195 Winthrop Eagles
47 Wisconsin Badgers
162 Wisconsin-(Green Bay) Phoenix
156 Wisconsin-(Milwaukee) Panthers
665 Wofford Terriers
387 Wright State Raiders
136 Wyoming Cowboys
169 Xavier Musketeers
326 Youngstown State Penguins

Done.


#### 2021-22

In [16]:
download_season(2021, ncaa_dict)

ncaa 2021
1281 Abilene Christian Wildcats
184 Air Force Falcons
174 Akron Zips
691 Alabama A&M Bulldogs
34 Alabama Crimson Tide
714 Alabama State Hornets
203 Albany Great Danes
307 Alcorn State Braves
322 American University Eagles
851 Appalachian State Mountaineers
122 Arizona State Sun Devils
40 Arizona Wildcats
78 Arkansas Razorbacks
119 Arkansas State Red Wolves
215 Arkansas-(Little Rock) Trojans
431 Arkansas-(Pine Bluff) Golden Lions
305 Army Black Knights
160 Auburn Tigers
448 Austin Peay Governors
325 Ball State Cardinals
173 Baylor Bears
3285 Bellarmine Knights
196 Belmont Bruins
436 Bethune-Cookman Wildcats
863 Binghamton Bearcats
171 Boise State Broncos
43 Boston College Eagles
438 Boston University Terriers
445 Bowling Green Falcons
141 Bradley Braves
159 Brigham Young Cougars
294 Brown Bears
854 Bryant University Bulldogs
123 Bucknell Bison
339 Buffalo Bulls
90 Butler Bulldogs
434 Cal Poly SLO Mustangs
869 Cal State Bakersfield Roadrunners
335 Cal State Fullerton Titans
682

81 UNLV Rebels
80 USC Trojans
849 USC Upstate Spartans
167 Utah State Aggies
67 Utah Utes
666 Utah Valley Wolverines
186 UTEP Miners
65 Valparaiso University
133 Vanderbilt Commodores
293 Vermont Catamounts
137 Villanova Wildcats
175 Virginia Cavaliers
442 Virginia Commonwealth Rams
284 Virginia Military Keydets
154 Virginia Tech Hokies
686 Wagner Seahawks
45 Wake Forest Demon Deacons
118 Washington Huskies
94 Washington State Cougars
380 Weber State Wildcats
84 West Virginia Mountaineers
677 Western Carolina Catamounts
861 Western Illinois Leathernecks
73 Western Kentucky Hilltoppers
297 Western Michigan Broncos
194 Wichita State Shockers
456 William & Mary Tribe
195 Winthrop Eagles
47 Wisconsin Badgers
162 Wisconsin-(Green Bay) Phoenix
156 Wisconsin-(Milwaukee) Panthers
665 Wofford Terriers
387 Wright State Raiders
136 Wyoming Cowboys
169 Xavier Musketeers
139 Yale Bulldogs
326 Youngstown State Penguins

Done.


In [17]:
driver.close()